In [3]:
import pandas as pd 
import numpy as np 

In [4]:
df = pd.read_csv('hotels_data.csv')
df

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.00,0,0,Check-Out,01/07/2015
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.00,0,0,Check-Out,01/07/2015
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.00,0,0,Check-Out,02/07/2015
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.00,0,0,Check-Out,02/07/2015
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.00,0,1,Check-Out,03/07/2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,City Hotel,0,23,2017,August,35,30,2,5,2,...,No Deposit,394.0,NaN,0,Transient,96.14,0,0,Check-Out,06/09/2017
119386,City Hotel,0,102,2017,August,35,31,2,5,3,...,No Deposit,9.0,NaN,0,Transient,225.43,0,2,Check-Out,07/09/2017
119387,City Hotel,0,34,2017,August,35,31,2,5,2,...,No Deposit,9.0,NaN,0,Transient,157.71,0,4,Check-Out,07/09/2017
119388,City Hotel,0,109,2017,August,35,31,2,5,2,...,No Deposit,89.0,NaN,0,Transient,104.40,0,0,Check-Out,07/09/2017


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

# Clean

In [6]:
# Ignore the row with ADR of 5000
df = df[df['adr']<1000]
# Ignore rows with undefined in DistributionChannel
df = df[df['distribution_channel']!= 'Undefined']
# Combine Undefined with SC
df['meal'].replace('Undefined','SC',inplace=True)
# Fill values with null Agent, Children with 0
df['agent'].fillna(0,inplace=True)
df['children'].fillna(0,inplace=True)


## Feature Engineering

In [7]:
# Create an assigned column that checks if the reserved room is the same as assigned
df['is_assigned'] = df['assigned_room_type'] == df['reserved_room_type']
# Total nights spent
df['total_nights'] = df['stays_in_week_nights'] + df['stays_in_weekend_nights']
# Total number of previous bookings
df['total_previous_bookings'] = df['previous_cancellations'] + df['previous_bookings_not_canceled']
# Reservation dates
df['reservation_date_day'] = pd.to_datetime(df['reservation_status_date']).dt.day
df['reservation_date_month'] = pd.to_datetime(df['reservation_status_date']).dt.month
# Total number of guests
df['total_number_of_guests'] = df['adults'] + df['children'] + df['babies']

c:\ProgramData\Anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '22/04/2015' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
c:\ProgramData\Anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '23/06/2015' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
c:\ProgramData\Anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '15/07/2015' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
c:\ProgramData\Anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '16/07/2015' in DD/MM/YYYY format. Provide format or specif

One Hot Encoding

In [8]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
encoder.fit(df[['arrival_date_month','meal','country']])
encoder.transform(df[['arrival_date_month','meal','country']]).toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

Encoding

In [9]:
from sklearn.preprocessing import LabelEncoder
cat_cols = ['hotel','arrival_date_month','meal','country','market_segment','distribution_channel','reserved_room_type','assigned_room_type','deposit_type','customer_type']
df[cat_cols] = df[cat_cols].apply(LabelEncoder().fit_transform)

y = df['is_canceled']

In [10]:
cols_to_keep = [
    'lead_time',
    'arrival_date_week_number',
    'arrival_date_day_of_month',
    'total_nights',
    'total_number_of_guests',
    'is_repeated_guest',
    'previous_cancellations',
    'previous_bookings_not_canceled',
    'total_previous_bookings',
    'is_assigned',
    'booking_changes',
    'agent',
    'days_in_waiting_list',
    'adr',
    'required_car_parking_spaces',
    'total_of_special_requests',
    'reservation_date_day',
    'reservation_date_month'
]
joined_lists = [*cols_to_keep,*cat_cols]
X = df[joined_lists]

In [31]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.preprocessing import StandardScaler

def train_algorithm_with_cross_validation(
        X: pd.DataFrame = None,
        y: pd.Series = None):
    '''
    This function takes the predictor and target variables and
    trains a Random Forest Regressor model across K folds. Using
    cross-validation, performance metrics will be output for each
    fold during training.

    Inputs: X-> Pandas Dataframe, predictor variables
            y-> Pandas series, target variable
    '''
    
    # list for holding the MAE scores across the K folds
    accuracy = [[],[],[],[]]
    SPLIT = 0.75
    K = 10
    for fold in range(0, K):

        # Instantiate RandomForest algorithm and StandardScaler
        scaler = StandardScaler()
        model = RandomForestClassifier()

        # Create training and test samples
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=SPLIT, random_state=42)

        # Scale X data, we scale the data because it helps the algorithm to converge
        # and helps the algorithm to not be greedy with large values
        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

        # Train model
        trained_model = model.fit(X_train, y_train)

        # Generate predictions on test sample
        y_pred = trained_model.predict(X_test)
    
        # Compute accuracy, using mean absolute error
        acc_score = accuracy_score(y_true=y_test, y_pred=y_pred)
        pre_score = precision_score(y_true=y_test, y_pred=y_pred)
        rec_score = recall_score(y_true=y_test, y_pred=y_pred)
        fone_score = f1_score(y_test,y_pred)
        for index, score in enumerate([acc_score,pre_score,rec_score,fone_score]):
            accuracy[index].append(score)
        
        # Displays the accuracy, precision, recall and f1-score for the current fold
        print(f"Fold {fold + 1}: Accuracy = {acc_score:.3f},\nPrecision = {pre_score:.3f},\nRecall = {rec_score:.3f},\nF1-Score = {fone_score:.3f}")
        
    
    # Computes average scores for accuracy, precision, recall and f1-score
    print(f"Average accuracy : {(sum(accuracy[0]) / len(accuracy[0])):.2f}")
    print(f"Average precision : {(sum(accuracy[1]) / len(accuracy[1])):.2f}")
    print(f"Average recall : {(sum(accuracy[2]) / len(accuracy[2])):.2f}")
    print(f"Average f1-score : {(sum(accuracy[3]) / len(accuracy[3])):.2f}")


In [32]:
train_algorithm_with_cross_validation(X,y)

Fold 1: Accuracy = 0.954,
Precision = 0.969,
Recall = 0.905,
F1-Score = 0.936
Fold 2: Accuracy = 0.952,
Precision = 0.966,
Recall = 0.903,
F1-Score = 0.933
Fold 3: Accuracy = 0.954,
Precision = 0.970,
Recall = 0.906,
F1-Score = 0.937
Fold 4: Accuracy = 0.955,
Precision = 0.970,
Recall = 0.906,
F1-Score = 0.937
Fold 5: Accuracy = 0.954,
Precision = 0.970,
Recall = 0.904,
F1-Score = 0.936
Fold 6: Accuracy = 0.952,
Precision = 0.966,
Recall = 0.902,
F1-Score = 0.933
Fold 7: Accuracy = 0.954,
Precision = 0.971,
Recall = 0.903,
F1-Score = 0.936
Fold 8: Accuracy = 0.955,
Precision = 0.971,
Recall = 0.907,
F1-Score = 0.938
Fold 9: Accuracy = 0.955,
Precision = 0.970,
Recall = 0.907,
F1-Score = 0.937
Fold 10: Accuracy = 0.954,
Precision = 0.968,
Recall = 0.906,
F1-Score = 0.936
Average accuracy : 0.95
Average precision : 0.97
Average recall : 0.90
Average f1-score : 0.94
